In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('E-Commerce').getOrCreate()

In [2]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True,header=True)

In [3]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [4]:
#Bringing that in the format for the machine learning algorithm]
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [7]:
#specifying which column are for the features and which are for label
assembler = VectorAssembler(
    inputCols = [ 'Avg Session Length', 'Time on App', 
                 'Time on Website', 'Length of Membership'], 
    outputCol = "features")

In [8]:
output = assembler.transform(data)

In [9]:
dataml = output.select('features','Yearly Amount Spent')
dataml.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [11]:
#splitting data into test train data set
train, test = dataml.randomSplit([0.7, 0.3])

train.describe().show()
test.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                344|
|   mean|  498.2791400078813|
| stddev|  76.82441908883628|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                156|
|   mean|  501.5961215806683|
| stddev|  84.76009979625177|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [12]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol = 'Yearly Amount Spent')

In [17]:
model = lr.fit(train)
print("Values for :\nIntercept:{}\nCoefficients:{}"\
      .format(model.intercept,model.coefficients))

Values for :
Intercept:-1052.7156291085996
Coefficients:[25.889314473,37.9718423042,0.564770150991,61.6390832491]


In [18]:
test_results = model.evaluate(test)
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  11.455596625779549|
| -2.6553858767242104|
|   3.911164099673101|
| -20.916865115326914|
| 0.33070328485962364|
|  22.849542616098233|
|  3.1811370281851623|
|   18.53913822466103|
|  3.4704017962248486|
|  -5.316758541503816|
|  -5.941573537188447|
| -1.5206236461336857|
|  1.6259995088694836|
|  0.5694856460740993|
| -6.9976155011890455|
| -16.994182339661222|
| -13.901065280860394|
| -5.0795662351866895|
|   4.463687117380914|
|-0.33699207467640235|
+--------------------+
only showing top 20 rows



In [19]:
print("The value of r-square :{}".format(test_results.r2))
print("The value of rmse :{}".format(test_results.rootMeanSquaredError))

The value of r-square :0.985557392076116
The value of rmse :10.153547855171935


In [20]:
#creatin unlabelled data from the test data 
unlabelled = test.select('features')

In [21]:
model.transform(unlabelled).show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.7377203726281...|450.32514557045033|
|[30.8794843441274...| 492.8619858615789|
|[31.0472221394875...| 388.5862350893483|
|[31.1239743499119...| 507.8639189550927|
|[31.2606468698795...|420.99592797209175|
|[31.2834474760581...| 568.9315468095692|
|[31.3091926408918...|429.53958081174846|
|[31.3123495994443...| 445.0522798032796|
|[31.4459724827577...| 481.4065631389037|
|[31.5171218025062...|281.23517919188953|
|[31.5257524169682...|449.90720034707033|
|[31.5761319713222...|  542.747207635462|
|[31.7366356860502...|495.30744674666244|
|[31.8293464559211...| 384.5828523419009|
|[31.8745516945853...|399.28285974745654|
|[31.9048571310136...|490.94403976247736|
|[31.9365486184489...| 441.1004501761886|
|[31.9453957483445...| 662.0994901728386|
|[31.9764800614612...| 326.1307589167193|
|[32.0047530203648...|464.08297319530584|
+--------------------+------------